# QENS data reduction

This notebook will guide you through the data reduction for the QENS experiment that you simulated with McStas yesterday.

The following is a basic outline of what this notebook will cover:

- Loading the NeXus files that contain the data
- Inspect/visualize the data contents
- How to convert the raw `time-of-flight` coordinate to something more useful ($\Delta E$)
- Write the results to file
- Process more than one pulse

In [ ]:
import numpy as np
import plopp as pp
import scippneutron as scn
import utils

import scipp as sc

## Load raw data

In [ ]:
# Optional: download the data
!wget -nc --no-verbose https://public.esss.dk/groups/scipp/dmsc-summer-school/qens/qens_elastic_1_pulse.h5
!wget -nc --no-verbose https://public.esss.dk/groups/scipp/dmsc-summer-school/qens/qens_known_quasi_elastic_1_pulse.h5
!wget -nc --no-verbose https://public.esss.dk/groups/scipp/dmsc-summer-school/qens/qens_unknown_quasi_elastic_1_pulse.h5

In [ ]:
fname = "qens_known_quasi_elastic_1_pulse.h5"
events = utils.load_qens(fname)

In [ ]:
events

## Inspect raw data

In [ ]:
events.hist(tof=100, y=100).plot()

## Mask bad region

In [ ]:
y = events.coords["y"]
mask_regions = sc.array(
        dims=["y"],
        values=[y.min().value, 0.26, 0.27, np.nextafter(y.max().value, np.inf)],
        unit=y.unit,
    )
binned_for_mask = events.bin(y=mask_regions)
mask = sc.array(dims=["y"], values=[False, True, False])
binned_for_mask.masks["bad_timing"] = mask
binned_for_mask

In [ ]:
binned_for_mask.hist().plot()

In [ ]:
binned_for_mask.hist(tof=100, y=100).plot()

## Transform to energy transfer

In [ ]:
def backscattered_l2(position, sample_position, analyzer_position):
    """
    Compute the length of the secondary flight path for backscattering off an analyzer.
    """
    # TODO Subtraction from Mad's script ?!
    return (
        sc.norm(position - analyzer_position)
        + sc.norm(analyzer_position - sample_position)
        - sc.scalar(0.03, unit="m")
    )


def wavelength_from_analyzer(analyzer_dspacing, analyzer_angle):
    """
    Compute the neutron wavelength after scattering from the analyzer's d-spacing.

    Assuming Bragg scattering in the analyzer, the wavelength is
        wavelength = 2 * d * sin(theta)

    Where
        d is the analyzer's d-spacing,
        theta is the scattering angle or equivalently, the tilt of the analyzer
              w.r.t. to the sample-analyzer axis.
    """
    # 2*theta is the angle between transmitted and scattered beam.
    return (
        2
        * analyzer_dspacing
        * sc.sin(sc.scalar(np.pi / 2, unit="rad") - analyzer_angle.to(unit="rad"))
    )


def final_energy(final_wavelength):
    """
    Compute the neutron energy after scattering.

    Uses
        final_energy = mn / 2 * final_speed**2
        final_speed = 2 * pi * hbar / mn / final_wavelength

    Where
        mn is the neutron mass,
        final_wavelength is the wavelength after scattering,
        final_speed is the speed after scattering.
    """
    return sc.to_unit(
        sc.constants.h**2 / 2 / sc.constants.neutron_mass / (final_wavelength**2),
        "meV",
    )

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.tof import energy_transfer_indirect_from_tof

graph = {
    **beamline(scatter=True),
    "L2": backscattered_l2,
    "final_wavelength": wavelength_from_analyzer,
    "final_energy": final_energy,
    "energy_transfer": energy_transfer_indirect_from_tof,
}
del graph["two_theta"]
del graph["scattered_beam"]
del graph["Ltotal"]
sc.show_graph(graph, simplified=True)

In [ ]:
def correct_tof(data):
    data = data.copy()
    # TODO according to Mads' script
    #  Instrument focuses at center of pulse, 2.86 ms / 2
    shift = sc.scalar(0.5 * 2.86, unit="ms")
    if "tof" in data.coords:
        data.coords["tof"] -= shift.to(unit=data.coords["tof"].unit)
    if data.bins and "tof" in data.bins.coords:
        data.bins.coords["tof"] -= shift.to(unit=data.bins.coords["tof"].bins.unit)
    return data

In [ ]:
def to_energy_transfer(data):
    data = correct_tof(data)
    return data.transform_coords("energy_transfer", graph=graph)

In [ ]:
unmasked = to_energy_transfer(events)
unmasked

In [ ]:
masked = to_energy_transfer(binned_for_mask)
masked

In [ ]:
masked_hist = masked.bins.concat().hist(energy_transfer=100)
unmasked_hist = unmasked.hist(energy_transfer=100)

In [ ]:
utils.add_variances(masked_hist)
utils.add_variances(unmasked_hist)

In [ ]:
pp.plot(
    {
        "unmasked": unmasked_hist,
        "masked": masked_hist,
    }
)

In [ ]:
# TODo do all 3 samples

## Save result to disk

In [ ]:
data = masked_hist.copy()
data.coords["energy_transfer"] = sc.midpoints(data.coords["energy_transfer"])
scn.io.save_xye("qens_energy_transfer_known_quasi_elastic_1_pulse.xye", data)

## Process data from 3 pulses

We now want to repeat the reduction, but using more than a single pulse to improve our statistics.

We begin by loading the run with 3 pulses.

In [ ]:
# Optional: download sample data
!wget -nc --no-verbose https://public.esss.dk/groups/scipp/dmsc-summer-school/qens/qens_elastic_3_pulse.h5
!wget -nc --no-verbose https://public.esss.dk/groups/scipp/dmsc-summer-school/qens/qens_known_quasi_elastic_3_pulse.h5
!wget -nc --no-verbose https://public.esss.dk/groups/scipp/dmsc-summer-school/qens/qens_unknown_quasi_elastic_3_pulse.h5

In [ ]:
fname = "qens_known_quasi_elastic_3_pulse.h5"
events_3_pulse = utils.load_qens(fname)
events_3_pulse

In [ ]:
events_3_pulse.hist(tof=100, y=100).plot()

### Exercise ?: fold the pulses

You are provided with a function that will perform the folding correctly,
assuming you supply it the correct inputs.

In [ ]:
utils.fold_pulses?

Make 3 pulse offsets, starting at 0, using 14 pulses per second.

In [ ]:
pulse_offsets = sc.arange("tof", 3.0) * sc.scalar(1.0 / 14.0, unit="s").to(unit="ms")
pulse_offsets

Make evenly-spaced pulse-edges.

In [ ]:
tmin = events_3_pulse.coords["tof"].min().value
tmax = np.nextafter(events_3_pulse.coords["tof"].max().value, np.inf)
edges = sc.linspace("tof", tmin, tmax, 4, unit="ms")
edges

In [ ]:
folded = utils.fold_pulses(events_3_pulse, edges, pulse_offsets)
folded.hist(tof=100, y=100).plot()

In [ ]:
folded

In [ ]:
binned_for_mask = folded.bin(y=mask_regions)
binned_for_mask.masks["bad_timing"] = mask
masked_3_pulse = to_energy_transfer(binned_for_mask)
masked_3_pulse_hist = masked_3_pulse.bins.concat().hist(energy_transfer=100)
utils.add_variances(masked_3_pulse_hist)
masked_3_pulse_hist.plot()

In [ ]:
n3 = masked_3_pulse_hist / sc.values(masked_3_pulse_hist.sum())
n1 = masked_hist / sc.values(masked_hist.sum())
pp.plot({'1': n1, '3': n3})